In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715206462274

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Patrick")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [17]:
import xgboost as xgb

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [22]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[22:24:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:20.09466                                                            
[1]	validation-rmse:19.07028                                                            
[2]	validation-rmse:18.11721                                                            
[3]	validation-rmse:17.22917                                                            
[4]	validation-rmse:16.40310                                                            
[5]	validation-rmse:15.63504                                                            
[6]	validation-rmse:14.92084                                                            
[7]	validation-rmse:14.26083                                                            
[8]	validation-rmse:13.64270                                                    

[89]	validation-rmse:6.53270                                                            
[90]	validation-rmse:6.53145                                                            
[91]	validation-rmse:6.53046                                                            
[92]	validation-rmse:6.52945                                                            
[93]	validation-rmse:6.52832                                                            
[94]	validation-rmse:6.52653                                                            
[95]	validation-rmse:6.52557                                                            
[96]	validation-rmse:6.52465                                                            
[97]	validation-rmse:6.52355                                                            
[98]	validation-rmse:6.52244                                                            
[99]	validation-rmse:6.52132                                                            
[100]	validation-rmse

[181]	validation-rmse:6.48005                                                           
[182]	validation-rmse:6.47978                                                           
[183]	validation-rmse:6.47951                                                           
[184]	validation-rmse:6.47922                                                           
[185]	validation-rmse:6.47900                                                           
[186]	validation-rmse:6.47850                                                           
[187]	validation-rmse:6.47833                                                           
[188]	validation-rmse:6.47805                                                           
[189]	validation-rmse:6.47776                                                           
[190]	validation-rmse:6.47746                                                           
[191]	validation-rmse:6.47722                                                           
[192]	validation-rmse

[273]	validation-rmse:6.45625                                                           
[274]	validation-rmse:6.45610                                                           
[275]	validation-rmse:6.45577                                                           
[276]	validation-rmse:6.45549                                                           
[277]	validation-rmse:6.45526                                                           
[278]	validation-rmse:6.45506                                                           
[279]	validation-rmse:6.45492                                                           
[280]	validation-rmse:6.45473                                                           
[281]	validation-rmse:6.45464                                                           
[282]	validation-rmse:6.45437                                                           
[283]	validation-rmse:6.45408                                                           
[284]	validation-rmse

[365]	validation-rmse:6.43846                                                           
[366]	validation-rmse:6.43829                                                           
[367]	validation-rmse:6.43824                                                           
[368]	validation-rmse:6.43815                                                           
[369]	validation-rmse:6.43795                                                           
[370]	validation-rmse:6.43769                                                           
[371]	validation-rmse:6.43753                                                           
[372]	validation-rmse:6.43742                                                           
[373]	validation-rmse:6.43741                                                           
[374]	validation-rmse:6.43694                                                           
[375]	validation-rmse:6.43671                                                           
[376]	validation-rmse

...

[506]	validation-rmse:6.43558                                                           
[507]	validation-rmse:6.43563                                                           
[508]	validation-rmse:6.43560                                                           
[509]	validation-rmse:6.43556                                                           
[510]	validation-rmse:6.43551                                                           
[511]	validation-rmse:6.43544                                                           
[512]	validation-rmse:6.43539                                                           
[513]	validation-rmse:6.43531                                                           
[514]	validation-rmse:6.43516                                                           
[515]	validation-rmse:6.43531                                                           
[516]	validation-rmse:6.43523                                                           
[517]	validation-rmse

[598]	validation-rmse:6.43170                                                           
[599]	validation-rmse:6.43154                                                           
[600]	validation-rmse:6.43154                                                           
[601]	validation-rmse:6.43146                                                           
[602]	validation-rmse:6.43149                                                           
[603]	validation-rmse:6.43146                                                           
[604]	validation-rmse:6.43120                                                           
[605]	validation-rmse:6.43116                                                           
[606]	validation-rmse:6.43120                                                           
[607]	validation-rmse:6.43119                                                           
[608]	validation-rmse:6.43112                                                           
[609]	validation-rmse

[690]	validation-rmse:6.42953                                                           
[691]	validation-rmse:6.42960                                                           
[692]	validation-rmse:6.42955                                                           
[693]	validation-rmse:6.42958                                                           
[694]	validation-rmse:6.42966                                                           
[695]	validation-rmse:6.42970                                                           
[696]	validation-rmse:6.42969                                                           
[697]	validation-rmse:6.42968                                                           
[698]	validation-rmse:6.42964                                                           
[699]	validation-rmse:6.42955                                                           
[700]	validation-rmse:6.42958                                                           
[701]	validation-rmse

[782]	validation-rmse:6.42916                                                           
[783]	validation-rmse:6.42920                                                           
[784]	validation-rmse:6.42928                                                           
[785]	validation-rmse:6.42926                                                           
[786]	validation-rmse:6.42932                                                           
[787]	validation-rmse:6.42929                                                           
[788]	validation-rmse:6.42938                                                           
[789]	validation-rmse:6.42942                                                           
[790]	validation-rmse:6.42942                                                           
[791]	validation-rmse:6.42944                                                           
[792]	validation-rmse:6.42950                                                           
[793]	validation-rmse

[62]	validation-rmse:6.42883                                                            
[63]	validation-rmse:6.42816                                                            
[64]	validation-rmse:6.42747                                                            
[65]	validation-rmse:6.42694                                                            
[66]	validation-rmse:6.42701                                                            
[67]	validation-rmse:6.42752                                                            
[68]	validation-rmse:6.42769                                                            
[69]	validation-rmse:6.42764                                                            
[70]	validation-rmse:6.42630                                                            
[71]	validation-rmse:6.42597                                                            
[72]	validation-rmse:6.42607                                                            
[73]	validation-rmse:

In [27]:
# mlflow.xgboost.autolog(disable=True)

```
learning_rate	0.10703770645309164
max_depth	22
min_child_weight	1.569626528283038
objective	reg:linear
reg_alpha	0.18055537091989357
reg_lambda	0.02281305329840515
seed	42
```

In [24]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.10703770645309164,
        'max_depth': 22,
        'min_child_weight': 1.569626528283038,
        'objective': 'reg:linear',
        'reg_alpha': 0.18055537091989357,
        'reg_lambda': 0.02281305329840515,
        'seed': 42
    }

    mlflow.log_params(best_params)

    # mlflow.xgboost.autolog()

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[06:30:20] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.28910
[1]	validation-rmse:17.60942
[2]	validation-rmse:16.13028
[3]	validation-rmse:14.82997
[4]	validation-rmse:13.69132
[5]	validation-rmse:12.69719
[6]	validation-rmse:11.83255
[7]	validation-rmse:11.08217
[8]	validation-rmse:10.43089
[9]	validation-rmse:9.86892
[10]	validation-rmse:9.38670
[11]	validation-rmse:8.97426
[12]	validation-rmse:8.62076
[13]	validation-rmse:8.31968
[14]	validation-rmse:8.06323
[15]	validation-rmse:7.84445
[16]	validation-rmse:7.65843
[17]	validation-rmse:7.50141
[18]	validation-rmse:7.36701
[19]	validation-rmse:7.25222
[20]	validation-rmse:7.15381
[21]	validation-rmse:7.07099
[22]	validation-rmse:6.99938
[23]	validation-rmse:6.93783
[24]	validation-rmse:6.88516
[25]	validation-rmse:6.84038
[26]	validation-rmse:6.80132
[27]	valid

[270]	validation-rmse:6.40870
[271]	validation-rmse:6.40847
[272]	validation-rmse:6.40806
[273]	validation-rmse:6.40775
[274]	validation-rmse:6.40689
[275]	validation-rmse:6.40667
[276]	validation-rmse:6.40631
[277]	validation-rmse:6.40632
[278]	validation-rmse:6.40609
[279]	validation-rmse:6.40603
[280]	validation-rmse:6.40561
[281]	validation-rmse:6.40527
[282]	validation-rmse:6.40520
[283]	validation-rmse:6.40488
[284]	validation-rmse:6.40433
[285]	validation-rmse:6.40397
[286]	validation-rmse:6.40372
[287]	validation-rmse:6.40344
[288]	validation-rmse:6.40315
[289]	validation-rmse:6.40307
[290]	validation-rmse:6.40278
[291]	validation-rmse:6.40211
[292]	validation-rmse:6.40197
[293]	validation-rmse:6.40182
[294]	validation-rmse:6.40170
[295]	validation-rmse:6.40133
[296]	validation-rmse:6.40105
[297]	validation-rmse:6.40092
[298]	validation-rmse:6.40074
[299]	validation-rmse:6.40041
[300]	validation-rmse:6.40014
[301]	validation-rmse:6.39981
[302]	validation-rmse:6.39969
[303]	vali

[544]	validation-rmse:6.35100
[545]	validation-rmse:6.35088
[546]	validation-rmse:6.35076
[547]	validation-rmse:6.35052
[548]	validation-rmse:6.35034
[549]	validation-rmse:6.35041
[550]	validation-rmse:6.35033
[551]	validation-rmse:6.35019
[552]	validation-rmse:6.34978
[553]	validation-rmse:6.34968
[554]	validation-rmse:6.34953
[555]	validation-rmse:6.34938
[556]	validation-rmse:6.34931
[557]	validation-rmse:6.34910
[558]	validation-rmse:6.34902
[559]	validation-rmse:6.34880
[560]	validation-rmse:6.34879
[561]	validation-rmse:6.34878
[562]	validation-rmse:6.34879
[563]	validation-rmse:6.34854
[564]	validation-rmse:6.34831
[565]	validation-rmse:6.34813
[566]	validation-rmse:6.34791
[567]	validation-rmse:6.34781
[568]	validation-rmse:6.34755
[569]	validation-rmse:6.34751
[570]	validation-rmse:6.34737
[571]	validation-rmse:6.34736
[572]	validation-rmse:6.34717
[573]	validation-rmse:6.34700
[574]	validation-rmse:6.34673
[575]	validation-rmse:6.34658
[576]	validation-rmse:6.34645
[577]	vali

[818]	validation-rmse:6.32005
[819]	validation-rmse:6.32013
[820]	validation-rmse:6.32012
[821]	validation-rmse:6.32002
[822]	validation-rmse:6.32002
[823]	validation-rmse:6.31993
[824]	validation-rmse:6.31980
[825]	validation-rmse:6.31979
[826]	validation-rmse:6.31966
[827]	validation-rmse:6.31950
[828]	validation-rmse:6.31951
[829]	validation-rmse:6.31943
[830]	validation-rmse:6.31926
[831]	validation-rmse:6.31910
[832]	validation-rmse:6.31887
[833]	validation-rmse:6.31871
[834]	validation-rmse:6.31853
[835]	validation-rmse:6.31847
[836]	validation-rmse:6.31846
[837]	validation-rmse:6.31836
[838]	validation-rmse:6.31830
[839]	validation-rmse:6.31813
[840]	validation-rmse:6.31762
[841]	validation-rmse:6.31748
[842]	validation-rmse:6.31741
[843]	validation-rmse:6.31738
[844]	validation-rmse:6.31741
[845]	validation-rmse:6.31740
[846]	validation-rmse:6.31735
[847]	validation-rmse:6.31719
[848]	validation-rmse:6.31700
[849]	validation-rmse:6.31694
[850]	validation-rmse:6.31674
[851]	vali

2022/05/25 06:32:35 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 5f9bd13703b34710ae68ec35c5052ab0. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'num_boost_round\', \'old_value\': None, \'new_value\': \'1000\'}, {\'key\': \'early_stopping_rounds\', \'old_value\': None, \'new_value\': \'50\'}, {\'key\': \'maximize\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'verbose_eval\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'custom_metric\', \'old_value\': None, \'new_value\': \'None\'}]\' for run ID=\'5f9bd13703b34710ae68ec35c5052ab0\'.")]')]


In [26]:
# From mlflow
logged_model = 'runs:/5f9bd13703b34710ae68ec35c5052ab0/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model

[06:40:17] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 5f9bd13703b34710ae68ec35c5052ab0

In [29]:
# Load logged model as an xgboost flavor
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

[06:43:42] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [30]:
# Make predictions with logged model
preds = xgboost_model.predict(valid)
preds[:10]

array([15.119596,  7.161393, 13.942863, 24.291872,  9.203283, 17.133688,
       12.20901 ,  8.938266,  8.91862 , 19.920511], dtype=float32)

In [31]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2022/05/25 06:59:44 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
/Users/patrickwalukagga/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
